<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/Chandelier_Exit_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance finta backtrader backtrader_plotting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import yfinance as yf
from datetime import datetime
import pandas as pd
from finta import TA
import backtrader as bt
from datetime import datetime
from backtrader_plotting import Bokeh
from backtrader_plotting.schemes import Tradimo

In [ ]:
symbol = 'BTC-USD'

#START DATE FOR yfinance

today = datetime.date.today()
count = 31
days = datetime.timedelta(count)
delta = today - days

start = datetime.datetime.strftime(delta, "%Y-%m-%d")
# print(start)

# END DATE FOR yfinance

end = today.strftime("%Y-%m-%d")
# print(end)

period = '5d'
interval = '60m'

df = yf.download(symbol, start=start, end=end, period=period, interval=interval)
df.reset_index(inplace=True)
# df['Date'] = df['Datetime'].dt.strftime('%Y/%m/%d')
df['Datetime'].dt.strftime('%Y/%m/%d')
df['Datetime'] = df['Datetime'].dt.strftime('%Y-%m-%d %H:%M').values
df

In [ ]:
atr_period = 22
atr = TA.ATR(df, period=atr_period)

#  Calculate chandelier exits
chandelier_info = TA.CHANDELIER(df, short_period=atr_period, long_period=atr_period, k=3) #default k=3

#  Add to price dataframe
df = pd.concat([df, atr, chandelier_info], axis=1, ignore_index=False)
df

In [ ]:
class ChandelierExit(bt.Indicator):

    lines = ('long', 'short')
    params = (('period', 22), ('multip', 3),)

    plotinfo = dict(subplot=False)

    def __init__(self):
        highest = bt.ind.Highest(self.data.high, period=self.p.period)
        lowest = bt.ind.Lowest(self.data.low, period=self.p.period)
        atr = self.p.multip * bt.ind.ATR(self.data, period=self.p.period)
        self.lines.long = highest - atr
        self.lines.short = lowest + atr

symbol = 'BTC-USD'
period = '5d'
interval = '60m'

data = bt.feeds.PandasData(dataname=yf.download(symbol, '2018-01-01', '2019-01-01'))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.add_signal(bt.SIGNAL_LONGSHORT, ChandelierExit)
cerebro.run()

In [ ]:
!curl -O https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=1631980233&period2=1663516233&interval=1d&events=history&includeAdjustedClose=true

In [ ]:
#NEW WAVES

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 16,9
rcParams['figure.facecolor'] = '#eeeeee'
plt.title('dummy')
plt.plot([1,3,2,4])
plt.close()

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

class ChandelierExit(bt.Strategy):

    # lines = ('long', 'short')
    params = (('period', 22), ('multip', 3),)

    plotinfo = dict(subplot=False)

    def __init__(self):
        highest = bt.ind.Highest(self.data.high, period=self.p.period)
        lowest = bt.ind.Lowest(self.data.low, period=self.p.period)
        atr = self.p.multip * bt.ind.ATR(self.data, period=self.p.period)
        self.lines.long = highest - atr
        self.lines.short = lowest + atr

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(ChandelierExit)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere

    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = '/content/BTC-USD.csv'

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2021, 9, 18),
        # Do not pass values before this date
        todate=datetime.datetime(2022, 9, 18),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(1000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # # Run over everything
    # cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
    cerebro.plot()